In [1]:
import os
import requests
import sqlite3

import pandas as pd

from urllib.parse import urlencode, quote
from dotenv import load_dotenv

In [2]:
load_dotenv()

ENCODING_API_KEY = os.environ["ENCODING_API_KEY"]
DECODING_API_KEY = os.environ["DECODING_API_KEY"]
BASE_URL = "http://apis.data.go.kr/B551011/{lang}Service2/"

DB_NAME = "../kto_data.db"

In [3]:
def get_data(endpoint:str, params:dict, lang:str="Kor"):
    url = BASE_URL.format(lang=lang) + endpoint + "?"+ urlencode(params)
    print()

    response = requests.get(url) 
    if response.status_code == 200:
        if "SERVICE ERROR" in response.text:
            print(response.text)
            return []
        
        try:
            data = response.json()["response"]["body"]

            if data["totalCount"] > 0:
                return data["items"]["item"]
        except Exception as e:
            print(e)
            print(response.text)
        
        return []
    else:
        print("API 호출 실패:", response.status_code)
        return []

In [4]:
def insert_into_db(insert_data:list, insert_query:str, create_query:str=None):
    conn = sqlite3.connect(DB_NAME)
    cur = conn.cursor()

    if create_query:
        cur.execute(create_query)

    cur.executemany(insert_query, insert_data)
    conn.commit()
    conn.close()

In [7]:
def drop_table(table_name:str):
    conn = sqlite3.connect(DB_NAME)
    cur = conn.cursor()
    cur.execute(f"DROP TABLE {table_name}")
    conn.commit()
    conn.close()

## 1. 코드 데이터 탐색

### 1\) 법정 지역 코드

In [12]:
endpoint = 'ldongCode2'
params = {
    "numOfRows": 300,
    "MobileOS": "ETC",
    "MobileApp": "ktrip-bot",
    "serviceKey": DECODING_API_KEY,
    "_type": "json",
    "lDongListYn": "Y"
}
lang = "Eng"
data = get_data(endpoint, params, lang)
data[:3]

[{'lDongRegnCd': '11',
  'lDongRegnNm': 'Seoul',
  'lDongSignguCd': '110',
  'lDongSignguNm': 'Jongno-gu',
  'rnum': 1},
 {'lDongRegnCd': '11',
  'lDongRegnNm': 'Seoul',
  'lDongSignguCd': '140',
  'lDongSignguNm': 'Jung-gu',
  'rnum': 2},
 {'lDongRegnCd': '11',
  'lDongRegnNm': 'Seoul',
  'lDongSignguCd': '170',
  'lDongSignguNm': 'Yongsan-gu',
  'rnum': 3}]

In [ ]:
# drop_table(endpoint)

In [13]:
create_query = f"""
CREATE TABLE IF NOT EXISTS {endpoint} (
    lDongRegnCd     VARCHAR(5),
    lDongRegnNm     TEXT,
    lDongSignguCd   VARCHAR(5),
    lDongSignguNm   TEXT,
    lang            VARCHAR(5)
)
"""
insert_query = f"""
INSERT INTO {endpoint} (lDongRegnCd, lDongRegnNm, lDongSignguCd, lDongSignguNm, lang) 
VALUES (:lDongRegnCd, :lDongRegnNm, :lDongSignguCd, :lDongSignguNm, '{lang}')
"""

insert_into_db(data, insert_query, create_query)

### 2\) 분류체계 코드

In [17]:
endpoint = 'lclsSystmCode2'
params = {
    "serviceKey": DECODING_API_KEY,
    "numOfRows": 300,
    "MobileOS": "ETC",
    "MobileApp": "ktrip-bot",
    "_type": "json",
    "lclsSystmListYn": "Y"
}
lang = "Eng"
data = get_data(endpoint, params, lang)
data[:3]

[{'lclsSystm1Cd': 'AC',
  'lclsSystm1Nm': 'Accommodation',
  'lclsSystm2Cd': 'AC01',
  'lclsSystm2Nm': 'Hotels',
  'lclsSystm3Cd': 'AC010100',
  'lclsSystm3Nm': 'Hotels',
  'rnum': 1},
 {'lclsSystm1Cd': 'AC',
  'lclsSystm1Nm': 'Accommodation',
  'lclsSystm2Cd': 'AC02',
  'lclsSystm2Nm': 'Condominiums',
  'lclsSystm3Cd': 'AC020100',
  'lclsSystm3Nm': 'Condominiums',
  'rnum': 2},
 {'lclsSystm1Cd': 'AC',
  'lclsSystm1Nm': 'Accommodation',
  'lclsSystm2Cd': 'AC02',
  'lclsSystm2Nm': 'Condominiums',
  'lclsSystm3Cd': 'AC020200',
  'lclsSystm3Nm': 'Serviced Residences',
  'rnum': 3}]

In [ ]:
# drop_table(endpoint)

In [18]:
create_query = f"""
CREATE TABLE IF NOT EXISTS {endpoint} (
    lclsSystm1Cd   VARCHAR(2),
    lclsSystm1Nm   TEXT,
    lclsSystm2Cd   VARCHAR(4),
    lclsSystm2Nm   TEXT,
    lclsSystm3Cd   VARCHAR(8),
    lclsSystm3Nm   TEXT,
    lang           VARCHAR(5)
)
"""
insert_query = f"""
INSERT INTO {endpoint} 
(lclsSystm1Cd, lclsSystm1Nm, lclsSystm2Cd, lclsSystm2Nm, lclsSystm3Cd, lclsSystm3Nm, lang)
VALUES 
(:lclsSystm1Cd, :lclsSystm1Nm, :lclsSystm2Cd, :lclsSystm2Nm, :lclsSystm3Cd, :lclsSystm3Nm, '{lang}')
"""

insert_into_db(data, insert_query, create_query)

## 2. 관광지 데이터 탐색

### 1\) 지역별 관광지 검색

In [155]:
endpoint = 'areaBasedList2'
params = {
    "numOfRows": 3,
    "MobileOS": "ETC",
    "MobileApp": "ktrip-bot",
    "_type": "json",
    "arrange": "C",
    "serviceKey": DECODING_API_KEY,
    "lDongRegnCd": "47",
    # "lclsSystm1": "VE",
    # "lclsSystm2": "VE01",
}
data = get_data(endpoint, params)
data

[{'addr1': '경상북도 경주시 천북면 물천리',
  'addr2': '944-7',
  'areacode': '35',
  'cat1': 'A02',
  'cat2': 'A0202',
  'cat3': 'A02020700',
  'contentid': '3501757',
  'contenttypeid': '12',
  'createdtime': '20250701165242',
  'firstimage': 'http://tong.visitkorea.or.kr/cms/resource/21/3501721_image2_1.jpg',
  'firstimage2': 'http://tong.visitkorea.or.kr/cms/resource/21/3501721_image3_1.jpg',
  'cpyrhtDivCd': 'Type3',
  'mapx': '129.2770278210',
  'mapy': '35.8688439662',
  'mlevel': '6',
  'modifiedtime': '20250701165259',
  'sigungucode': '2',
  'tel': '',
  'title': '경주시 자전거공원',
  'zipcode': '38038',
  'lDongRegnCd': '47',
  'lDongSignguCd': '130',
  'lclsSystm1': 'VE',
  'lclsSystm2': 'VE03',
  'lclsSystm3': 'VE030500'},
 {'addr1': '경상북도 의성군 다인면 대곡사길 80',
  'addr2': '',
  'areacode': '35',
  'cat1': 'A02',
  'cat2': 'A0201',
  'cat3': 'A02010800',
  'contentid': '127360',
  'contenttypeid': '12',
  'createdtime': '20040226090000',
  'firstimage': 'http://tong.visitkorea.or.kr/cms/resource/1

### 2\) 관광지 키워드 검색

In [138]:
endpoint = 'searchKeyword2'
params = {
    "numOfRows": 300,
    "MobileOS": "ETC",
    "MobileApp": "ktrip-bot",
    "_type": "json",
    "arrange": "C",
    "keyword": "카페",
    "serviceKey": DECODING_API_KEY,
    "lDongRegnCd": "47",
    # "lclsSystm1": "VE",
    # "lclsSystm2": "VE01",
}
data = get_data(endpoint, params)
data


{"response": {"header":{"resultCode":"0000","resultMsg":"OK"},"body": {"items": {"item":[{"addr1":"경상북도 경주시 동해안로 1488","addr2":"","zipcode":"38123","areacode":"35","cat1":"A05","cat2":"A0502","cat3":"A05020900","contentid":"2905411","contenttypeid":"39","createdtime":"20221110172135","firstimage":"http://tong.visitkorea.or.kr/cms/resource/93/2905393_image2_1.jpg","firstimage2":"http://tong.visitkorea.or.kr/cms/resource/93/2905393_image3_1.jpg","cpyrhtDivCd":"Type3","mapx":"129.4847381802","mapy":"35.7454007193","mlevel":"6","modifiedtime":"20250626155850","sigungucode":"2","tel":"","title":"감포이견대카페","lDongRegnCd":"47","lDongSignguCd":"130","lclsSystm1":"FD","lclsSystm2":"FD05","lclsSystm3":"FD050100"},{"addr1":"경상북도 포항시 북구 동해대로 3290","addr2":"","zipcode":"37510","areacode":"35","cat1":"A05","cat2":"A0502","cat3":"A05020900","contentid":"2842542","contenttypeid":"39","createdtime":"20220824100356","firstimage":"http://tong.visitkorea.or.kr/cms/resource/36/2842536_image2_1.jpg","firstim

[{'addr1': '경상북도 경주시 동해안로 1488',
  'addr2': '',
  'zipcode': '38123',
  'areacode': '35',
  'cat1': 'A05',
  'cat2': 'A0502',
  'cat3': 'A05020900',
  'contentid': '2905411',
  'contenttypeid': '39',
  'createdtime': '20221110172135',
  'firstimage': 'http://tong.visitkorea.or.kr/cms/resource/93/2905393_image2_1.jpg',
  'firstimage2': 'http://tong.visitkorea.or.kr/cms/resource/93/2905393_image3_1.jpg',
  'cpyrhtDivCd': 'Type3',
  'mapx': '129.4847381802',
  'mapy': '35.7454007193',
  'mlevel': '6',
  'modifiedtime': '20250626155850',
  'sigungucode': '2',
  'tel': '',
  'title': '감포이견대카페',
  'lDongRegnCd': '47',
  'lDongSignguCd': '130',
  'lclsSystm1': 'FD',
  'lclsSystm2': 'FD05',
  'lclsSystm3': 'FD050100'},
 {'addr1': '경상북도 포항시 북구 동해대로 3290',
  'addr2': '',
  'zipcode': '37510',
  'areacode': '35',
  'cat1': 'A05',
  'cat2': 'A0502',
  'cat3': 'A05020900',
  'contentid': '2842542',
  'contenttypeid': '39',
  'createdtime': '20220824100356',
  'firstimage': 'http://tong.visitkorea.o

### 3\) 행사 정보 조회

In [ ]:
endpoint = 'searchFestival2'
params = {
    "numOfRows": 3,
    "MobileOS": "ETC",
    "MobileApp": "ktrip-bot",
    "_type": "json",
    "arrange": "C",
    "eventStartDate": "20250701",
    "serviceKey": DECODING_API_KEY,
    "lDongRegnCd": "47"
}
data = get_data(endpoint, params)
data

### 4\) 숙박 정보 조회

In [154]:
endpoint = 'searchStay2'
params = {
    "numOfRows": 3,
    "MobileOS": "ETC",
    "MobileApp": "ktrip-bot",
    "_type": "json",
    "arrange": "C",
    "serviceKey": DECODING_API_KEY,
    "lDongRegnCd": "47"
}
data = get_data(endpoint, params)
data

[{'addr1': '경상북도 경주시 태종로699번길 31',
  'addr2': '',
  'areacode': '35',
  'sigungucode': '2',
  'cat1': 'B02',
  'cat2': 'B0201',
  'cat3': 'B02011100',
  'contentid': '2950401',
  'contenttypeid': '32',
  'createdtime': '20230217160233',
  'firstimage': 'http://tong.visitkorea.or.kr/cms/resource/10/2949210_image2_1.jpg',
  'firstimage2': 'http://tong.visitkorea.or.kr/cms/resource/10/2949210_image3_1.jpg',
  'cpyrhtDivCd': 'Type1',
  'mapx': '129.2046571650',
  'mapy': '35.8415095729',
  'mlevel': '6',
  'modifiedtime': '20250626162555',
  'tel': '',
  'title': '블라우[한국관광 품질인증/Korea Quality]',
  'zipcode': '38157',
  'lDongRegnCd': '47',
  'lDongSignguCd': '130',
  'lclsSystm1': 'AC',
  'lclsSystm2': 'AC06',
  'lclsSystm3': 'AC060200'},
 {'addr1': '경상북도 경주시 국당2길 12-17',
  'addr2': '',
  'areacode': '35',
  'sigungucode': '2',
  'cat1': 'B02',
  'cat2': 'B0201',
  'cat3': 'B02011600',
  'contentid': '2706608',
  'contenttypeid': '32',
  'createdtime': '20210219201904',
  'firstimage': 'htt

### 5\) 관광지 Overview 조회

In [ ]:
endpoint = 'detailCommon2'
params = {
    "MobileOS": "ETC",
    "MobileApp": "ktrip-bot",
    "_type": "json",
    "contentId": "2905411",
    "numOfRows": 1,
    "serviceKey": DECODING_API_KEY,
}
data = get_data(endpoint, params)
data

[{'contentid': '2905411',
  'contenttypeid': '39',
  'title': '감포이견대카페',
  'createdtime': '20221110172135',
  'modifiedtime': '20250626155850',
  'tel': '',
  'telname': '',
  'homepage': '',
  'firstimage': 'http://tong.visitkorea.or.kr/cms/resource/93/2905393_image2_1.jpg',
  'firstimage2': 'http://tong.visitkorea.or.kr/cms/resource/93/2905393_image3_1.jpg',
  'cpyrhtDivCd': 'Type3',
  'areacode': '35',
  'sigungucode': '2',
  'lDongRegnCd': '47',
  'lDongSignguCd': '130',
  'lclsSystm1': 'FD',
  'lclsSystm2': 'FD05',
  'lclsSystm3': 'FD050100',
  'cat1': 'A05',
  'cat2': 'A0502',
  'cat3': 'A05020900',
  'addr1': '경상북도 경주시 동해안로 1488',
  'addr2': '',
  'zipcode': '38123',
  'mapx': '129.4847381802',
  'mapy': '35.7454007193',
  'mlevel': '6',
  'overview': '감포이견대카페는 경상북도 경주시 감포읍 대본리에 있다. 이국적인 멋을 풍기는 외관과 고급스러운 분위기를 연출하는 인테리어가 눈에 띈다. 내부에는 단체석이 마련되어 있어 각종 모임을 하기 좋다. 대표 메뉴는 티라미수다. 이 밖에 아보카도 크림라떼, 아보카 스무디, 아메리카노, 카페라떼, 콜드브루 등 여러 가지 음료가 준비되어 있다. 동경주 IC에서 가깝고, 주변에 경주문무대왕릉과 나정고운모래해변이 있다.'}]

### 6\) 관광지 메뉴, 주차, 오픈 시간 등 조회

In [150]:
endpoint = 'detailIntro2'
params = {
    "MobileOS": "ETC",
    "MobileApp": "ktrip-bot",
    "_type": "json",
    "contentId": "2905411",
    "contentTypeId": "39",
    "numOfRows": 1,
    "serviceKey": DECODING_API_KEY,
}
data = get_data(endpoint, params)
data

[{'contentid': '2905411',
  'contenttypeid': '39',
  'seat': '',
  'kidsfacility': '0',
  'firstmenu': '티라미수',
  'treatmenu': '아보카도 크림라떼, 아메리카노, 카페라떼, 콜드브루 등',
  'smoking': '',
  'packing': '가능',
  'infocenterfood': '054-777-1333',
  'scalefood': '',
  'parkingfood': '있음(무료)',
  'opendatefood': '',
  'opentimefood': '09:00~21:00 (동절기 10:00~20:30)',
  'restdatefood': '연중무휴',
  'discountinfofood': '',
  'chkcreditcardfood': '가능',
  'reservationfood': '가능',
  'lcnsno': '20210542876'}]

In [151]:
endpoint = 'detailInfo2'
params = {
    "MobileOS": "ETC",
    "MobileApp": "ktrip-bot",
    "_type": "json",
    "contentId": "2905411",
    "contentTypeId": "39",
    "numOfRows": 1,
    "serviceKey": DECODING_API_KEY,
}
data = get_data(endpoint, params)
data

[{'contentid': '2905411',
  'contenttypeid': '39',
  'serialnum': '0',
  'infoname': '화장실',
  'infotext': '있음',
  'fldgubun': '1'}]

In [153]:
endpoint = 'detailImage2'
params = {
    "MobileOS": "ETC",
    "MobileApp": "ktrip-bot",
    "_type": "json",
    "contentId": "2905411",
    "imageYN": "Y",
    "numOfRows": 10,
    "serviceKey": DECODING_API_KEY,
}
data = get_data(endpoint, params)
data

[{'contentid': '2905411',
  'originimgurl': 'http://tong.visitkorea.or.kr/cms/resource/94/2905394_image2_1.jpg',
  'imgname': '경북_경주시_감포이견대카페_음식_1',
  'smallimageurl': 'http://tong.visitkorea.or.kr/cms/resource/94/2905394_image3_1.jpg',
  'cpyrhtDivCd': 'Type3',
  'serialnum': '2905394_2'},
 {'contentid': '2905411',
  'originimgurl': 'http://tong.visitkorea.or.kr/cms/resource/95/2905395_image2_1.jpg',
  'imgname': '경북_경주시_감포이견대카페_음식_2',
  'smallimageurl': 'http://tong.visitkorea.or.kr/cms/resource/95/2905395_image3_1.jpg',
  'cpyrhtDivCd': 'Type3',
  'serialnum': '2905395_4'},
 {'contentid': '2905411',
  'originimgurl': 'http://tong.visitkorea.or.kr/cms/resource/96/2905396_image2_1.jpg',
  'imgname': '경북_경주시_감포이견대카페_음식_3',
  'smallimageurl': 'http://tong.visitkorea.or.kr/cms/resource/96/2905396_image3_1.jpg',
  'cpyrhtDivCd': 'Type3',
  'serialnum': '2905396_1'},
 {'contentid': '2905411',
  'originimgurl': 'http://tong.visitkorea.or.kr/cms/resource/97/2905397_image2_1.jpg',
  'imgname'